In [1]:
import numpy as np
%reload_ext autoreload
%autoreload 2

# API Requests
import requests

# Request Wrangling
import os
from os.path import basename, exists
import json

# Data wrangling
import pandas as pd

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Machine learning model
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline, make_pipeline

# Check work
from sklearn.utils.validation import check_is_fitted

from scryfall_api import ScryfallAPI, ScryfallDataWrangler

# Supress warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [65]:
scryfall = ScryfallAPI()

print(f"API Response type: {type(scryfall.response)}")
print(f"API Request status code: {scryfall.response.status_code}")

API Response type: <class 'requests.models.Response'>
API Request status code: 200


In [66]:
scryfall.get_dataset()

{'success': True,
 'message': "Successfully downloaded 'all-cards-20221224221638.json'."}

In [80]:
df = scryfall.wrangler.wrangle()

print(f"df shape: {df.shape}")
print()
print(df.info())
print()
df.head()

df shape: (25624, 54)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25624 entries, 0 to 391494
Data columns (total 54 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     25624 non-null  object        
 1   name                   25624 non-null  object        
 2   released_at            25624 non-null  datetime64[ns]
 3   layout                 25624 non-null  object        
 4   mana_cost              25179 non-null  object        
 5   cmc                    25624 non-null  float64       
 6   type_line              25624 non-null  object        
 7   color_identity         25624 non-null  object        
 8   keywords               25624 non-null  object        
 9   legalities             25624 non-null  object        
 10  set_name               25624 non-null  object        
 11  set_type               25624 non-null  object        
 12  rarity                 25624 non-nul

,id,name,released_at,layout,mana_cost,cmc,type_line,color_identity,keywords,legalities,...,commander_legal,brawl_legal,historicbrawl_legal,alchemy_legal,paupercommander_legal,duel_legal,oldschool_legal,premodern_legal,has_flavor_text,price_usd
0,0000579f-7b35-4ed3-b44c-db2a538066fe,Fury Sliver,2006-10-06,normal,{5}{R},6.0,Creature — Sliver,[R],[],"{'standard': 'not_legal', 'future': 'not_legal...",...,legal,not_legal,not_legal,not_legal,restricted,legal,not_legal,not_legal,True,0.36
1,00006596-1166-4a79-8443-ca9f82e6db4e,Kor Outfitter,2009-10-02,normal,{W}{W},2.0,Creature — Kor Soldier,[W],[],"{'standard': 'not_legal', 'future': 'not_legal...",...,legal,not_legal,not_legal,not_legal,legal,legal,not_legal,not_legal,True,0.08
4,0000cd57-91fe-411f-b798-646e965eec37,Siren Lookout,2017-09-29,normal,{2}{U},3.0,Creature — Siren Pirate,[U],"[Flying, Explore]","{'standard': 'not_legal', 'future': 'not_legal...",...,legal,not_legal,legal,not_legal,legal,legal,not_legal,not_legal,False,0.04
17,0001f1ef-b957-4a55-b47f-14839cdbab6f,Venerable Knight,2019-10-04,normal,{W},1.0,Creature — Human Knight,[W],[],"{'standard': 'not_legal', 'future': 'not_legal...",...,legal,not_legal,legal,not_legal,restricted,legal,not_legal,not_legal,True,0.11
22,0002ab72-834b-4c81-82b1-0d2760ea96b0,Mystic Skyfish,2020-07-03,normal,{2}{U},3.0,Creature — Fish,[U],[],"{'standard': 'not_legal', 'future': 'not_legal...",...,legal,not_legal,legal,not_legal,legal,legal,not_legal,not_legal,True,0.07


# Machine Learning Price Predictor model for MTG cards

---
## 0. Untap - Project info

#### In this project, the goal is to take

#### Set restrictions:
#### <font color = "Green"> - Only non-online cards and sets will be analysed
#### - Sets that are comprised of masterpieces, Secret Lair Drops, Friday Night Magic Cards, Judge Gift promos and other exclusive, high-end or niche cards will not be analysed
#### <font color = "Green"> - Only cards that are not part of an "Un-" set will be analysed
#### - Retail price as specified by TCGPlayer
#### - Only the price of each card at the day of its respective set or product launch will be regarded
#### <font color = "Green"> - Only cards that aren't reprints will be analysed
#### <font color = "Green"> - Only cards that aren't tokens will be analysed
#### - Cards that are of the type Conspiracy, Emblem, Hero, Phenomenon, Plane, Scheme or Vanguard won't be analysed.
#### <font color = "Green"> - Basic Lands won't be analysed
#### <font color = "Green"> - Cards on The Reserved List won't be analysed
#### <font color = "Green"> - Oversized cards won't be analysed



---
## 1. Upkeep - Importing the Libraries

---
## 2. Draw - Importing the Data

---
## 3. First Main Phase - Treating df